# Tugas 2

## Distraksi
Distraksi merupakan transformasi kolom menjadi type data lain. Tujuannya untuk membuat data menjadi lebih optimal yang nantinya memudahkan untuk di analisis

In [ ]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 

In [ ]:
df = pd.read_csv("https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv")
df

## Equal-Width Intervals

Equal-width intervals adalah distraksi yang membagi data numerik menjadi beberapa kelompok dengan lebar kelompok yang kurang lebih sama besar.

Mengambil sample data

In [ ]:
data_sepal_length = df[["sepal.length"]]
data_sepal_width = df[["sepal.width"]]
data_petal_length = df[["petal.length"]]
data_petal_width = df[["petal.width"]]

Pandas Menyediakan method cut yang digunakan untuk mencari interval menggunakan metode Equal-width Intervals

In [ ]:
def cut(col, k):
    intervals = pd.cut(df[col], k).value_counts().index.to_list()
    print(intervals)
    return [[interval.left, interval.right] for interval in intervals]

In [ ]:
def toCategory(list_interval, col):
    # get length interval
    length = len(list_interval)

    # Sorting interval
    sort_interval = np.sort(list_interval, axis=0)
    # Get Category from intervala
    categories = np.array([chr(65+i) for i in range(length)])[:, None]
    # combine into interval data
    intervals = np.hstack((sort_interval, categories))

    # Operate All data
    newCol = []
    for i, row in df.iterrows():
        data = row[col]
        for interval in intervals:
            if data >= interval[0].astype(float) and data <= interval[1].astype(float):
                newCol.append(interval[2])
                break
    # return new coloumn category
    return np.array(newCol, dtype=str)


Mencari interval
sebagai contoh disini dibagi menjadi 3 bagian

In [ ]:
interval_sepal_length = cut("sepal.length", 3)
interval_sepal_width = cut("sepal.width", 3)
interval_petal_length = cut("petal.length", 3)
interval_petal_width = cut("petal.width", 3)

print("interval sepal.length :", interval_sepal_length)
print("interval sepal.width  :", interval_sepal_width)
print("interval petal.length :", interval_petal_length)
print("interval petal.width  :", interval_petal_width)

Tampilkan hasil dari pembagian category

In [ ]:
data_sepal_length["cateogry"] = toCategory(interval_sepal_length, "sepal.length")
data_sepal_width["cateogry"] = toCategory(interval_sepal_width, "sepal.width")
data_petal_length["cateogry"] = toCategory(interval_petal_length, "petal.length")
data_petal_width["cateogry"] = toCategory(interval_petal_width, "petal.width")

display(data_sepal_length)
display(data_sepal_width)
display(data_petal_length)
display(data_petal_width)

## Equal-frequency Intervals

Equal-frequency intervals adalah distraksi yang membagi data numerik menjadi beberapa kelompok dengan jumlah anggota yang kurang lebih sama besar.

In [ ]:
data_sepal_length = df[["sepal.length"]]
data_sepal_width = df[["sepal.width"]]
data_petal_length = df[["petal.length"]]
data_petal_width = df[["petal.width"]]

Pandas menyediakan method qcut untuk mencri nilai interval dari Equal-frequency Intervals

In [ ]:
def qcut(col, k):
    intervals = pd.qcut(df[col], k).value_counts().index.to_list()
    return [[interval.left, interval.right] for interval in intervals]

Mencari interval
sebagai contoh disini dibagi menjadi 3 bagian

In [ ]:
interval_sepal_length = qcut("sepal.length", 3)
interval_sepal_width = qcut("sepal.width", 3)
interval_petal_length = qcut("petal.length", 3)
interval_petal_width = qcut("petal.width", 3)

print("interval sepal.length :", interval_sepal_length)
print("interval sepal.width  :", interval_sepal_width)
print("interval petal.length :", interval_petal_length)
print("interval petal.width  :", interval_petal_width)

Tampilkan hasil dari pembagian category

In [ ]:
data_sepal_length["cateogry"] = toCategory(interval_sepal_length, "sepal.length")
data_sepal_width["cateogry"] = toCategory(interval_sepal_width, "sepal.width")
data_petal_length["cateogry"] = toCategory(interval_petal_length, "petal.length")
data_petal_width["cateogry"] = toCategory(interval_petal_width, "petal.width")

display(data_sepal_length)
display(data_sepal_width)
display(data_petal_length)
display(data_petal_width)

## Entropy

Entropy merupakan informasi yang menggambarkan seberapa konsisten pemisah akan cocok dengan pengklasifikasi

 Membuat sample data untuk di analisi

In [ ]:
sample = df[["sepal.length"]]
sample.describe()

Membuat category random untuk semua data

In [ ]:
np.random.seed(0)
sample["category"] = np.where(np.random.choice(2, sample.shape[0]) < 1, "A", "B" )
sample
sample.shape[0]

Membut fungsi getOverollCategory digunakan untuk menghitung data keseluruhan yang nantinya digunakan untuk menghitung entropy

In [ ]:
def getOverollCategory(col):
    group = sample.loc[:, :].groupby("category").count()
    a = group.loc["A", col]
    b = group.loc["B", col]
    print(a,b, group)
    return (a, b, a+b)

Fungsi splitter digunakan untuk membuat split antara value yang telah di tentukan lalu mengembalikan data yang telah di pisahkan

In [ ]:
def splitter(value:float, col:str)->tuple:
    # get data less and greater from value
    less = sample[sample[col] <= value]
    greater = sample[sample[col] > value]

    # calculate into category for each data
    less_group = less.loc[:, :].groupby("category").count()
    print(less_group)
    greater_group = greater.loc[:, :].groupby("category").count()

    # get value based on category
    less_category_A = less_group.loc["A", col]
    print(less_category_A)

    less_category_B = less_group.loc["B", col]
    greater_category_A = greater_group.loc["A", col]
    greater_category_B = greater_group.loc["B", col]

    return (
        [less_category_A, less_category_B, less_category_A + less_category_B],
        [greater_category_A, greater_category_B, greater_category_A + greater_category_B]
    )



Buat Fungsi entropy untuk mencari nilai entropy

In [ ]:
def entropy(d):
    r1 = (d[0] / d[2]) * np.log2(d[0] / d[2])
    r2 = (d[1] / d[2]) * np.log2(d[1] / d[2])
    return np.sum([r1, r2]) * -1

Membuat fungsi info dan gain

In [ ]:
def info(d):
    r1 = (d[0][2] / sample.shape[0])  * entropy(d[0])
    r2 = (d[1][2] / sample.shape[0]) * entropy(d[1])
    return r1 + r2

Fungsi gain untuk menghitung selsih antara entropy awal dengan yang baru

In [ ]:
def gain(Einitial, Enew):
    return Einitial - Enew

Membuat Dinitial

In [ ]:
D = getOverollCategory("sepal.length")
entropy_d = entropy(D)
print(D)
print(entropy_d)

Melakukan beberapa tes split untuk mencri informasi yang terbaik

*Split 1: 4.4*

In [ ]:
split1 = splitter(4.4, "sepal.length")
info_split1 = info(split1)
gain(entropy_d, info_split1)

*split 2: 5.5*

In [ ]:
split2 = splitter(5.5, "sepal.length")
info_split2 = info(split2)
gain(entropy_d, info_split2)

*split 3: 7.0*

In [ ]:
split3 = splitter(7.0, "sepal.length")
info_split3 = info(split3)
gain(entropy_d, info_split3)

Dari keseluruhan split yang telah di hitung. maka split terbaik adalah split 3 yang memberikan keuntungan informasi sebesar 0.0005490214732508658